In [2]:
pip install numpy

  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

  Using cached pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install nltk

Processing /home/wizardcalidad/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155/nltk-3.5-py3-none-any.whl
     |████████████████████████████████| 675 kB 448 kB/s eta 0:00:01
  Using cached joblib-0.16.0-py3-none-any.whl (300 kB)
     |████████████████████████████████| 70 kB 242 kB/s eta 0:00:01
     |████████████████████████████████| 82 kB 103 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sklearn

Processing /home/wizardcalidad/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.2-cp38-cp38-manylinux1_x86_64.whl (6.8 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached scipy-1.5.2-cp38-cp38-manylinux1_x86_64.whl (25.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.naive_bayes import BernoulliNB 

In [10]:
data = pd.read_csv('imdb_labelled.txt', sep='\t', names=[ 'reviews', 'score'])
data.head()

,reviews,score
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [11]:
def get_sentiments(data):
  if data['score'] == 0:
    return 'negative'
  elif data['score'] == 1:
    return 'positive'
  else:
    return 'indifferent'

data['sentiments'] = data.apply(get_sentiments, axis=1)
data.head()

,reviews,score,sentiments
0,"A very, very, very slow-moving, aimless movie ...",0,negative
1,Not sure who was more lost - the flat characte...,0,negative
2,Attempting artiness with black & white and cle...,0,negative
3,Very little music or anything to speak of.,0,negative
4,The best scene in the movie was when Gerardo i...,1,positive


In [12]:
data.isnull().sum()

reviews       0
score         0
sentiments    0
dtype: int64

In [13]:
data['length'] = data['reviews'].apply(lambda x: len(str(x).split(' ')))
data.head()

,reviews,score,sentiments,length
0,"A very, very, very slow-moving, aimless movie ...",0,negative,15
1,Not sure who was more lost - the flat characte...,0,negative,21
2,Attempting artiness with black & white and cle...,0,negative,33
3,Very little music or anything to speak of.,0,negative,10
4,The best scene in the movie was when Gerardo i...,1,positive,23


In [14]:
data['length'].unique()

array([  15,   21,   33,   10,   23,   22,    5,   17,   12,    8,   13,
          6,   18,   25,   19,  868,   14,   16,    9,    7,   26,   35,
         11,    4,   27,   28,    3,   20,   37,   24,   36,  201, 1393,
         49,   30,  306,   31,   32,   44,   56,   45,   34,   38,   64,
         58,   40,   47,   29,   73,   41,   39,   55,   53,   42,  800])

In [15]:
X = data['reviews']
y = data['score']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wizardcalidad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/wizardcalidad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import TfidfTransformer
lm = WordNetLemmatizer()

In [19]:
pipeNB = Pipeline([
      ('bow',CountVectorizer(stop_words='english')),
      ('tfidf',TfidfTransformer()),
      ('classifier',BernoulliNB())
])

In [20]:
pipeNB.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()), ('classifier', BernoulliNB())])

In [21]:
pred = pipeNB.predict(X_test) #predict testing data

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.86      0.52      0.65        60
           1       0.62      0.91      0.74        53

    accuracy                           0.70       113
   macro avg       0.74      0.71      0.69       113
weighted avg       0.75      0.70      0.69       113



In [22]:
from statistics import mean, stdev 
from sklearn.model_selection import StratifiedKFold 


accuracy = []
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=100) 
skf.get_n_splits(X,y) 

for train_index, test_index in skf.split(X, y):
  X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index] 
  y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

  pipeNB.fit(X_train_fold, y_train_fold)
  result = pipeNB.score(X_test_fold, y_test_fold)
  accuracy.append(result)


# Print the output
print('List of possible accuracy:', accuracy) 
print('\nMaximum Accuracy That can be obtained from this model is:', 
  max(accuracy)*100, '%') 
print('\nMinimum Accuracy:', 
  min(accuracy)*100, '%') 
print('\nOverall Accuracy:', 
  mean(accuracy)*100, '%') 
print('\nStandard Deviation is:', stdev(accuracy)) 

List of possible accuracy: [0.76, 0.8133333333333334, 0.84, 0.7066666666666667, 0.7866666666666666, 0.7066666666666667, 0.72, 0.72, 0.7567567567567568, 0.6756756756756757]

Maximum Accuracy That can be obtained from this model is: 84.0 %

Minimum Accuracy: 67.56756756756756 %

Overall Accuracy: 74.85765765765765 %

Standard Deviation is: 0.052345655361288296
